In [ ]:
import os
import sys
import joblib
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_utils/helper') 
%matplotlib notebook

In [ ]:
import warnings
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go

from statistics import mean
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from read_mongo_collection import HelperToReadMongo
from data_processing import DataProcessor
from network_activities import Network

from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

from tbats import TBATS

sns.set_theme(style="darkgrid")
warnings.filterwarnings("ignore")

# See complete data
# pd.set_option('max_columns', None)
# pd.set_option("max_rows", None)
# np.set_printoptions(threshold=np.inf)

In [ ]:
helper_mongo = HelperToReadMongo()
data_processor = DataProcessor()
network = Network()

In [ ]:
# Specify names of all the databases here
database = 'PowerTAC2021_CUP_Data_Collection'
collection1 = 'DistributionTransaction_and_Report_Info'

In [ ]:
result_path = '/mnt/d/PowerTAC/PowerTAC2021/experiments/forecasting_check/benchmarking_consumption_forecast/'

In [ ]:
db_demand = helper_mongo.query_to_mongo(database, collection1, server_ip='192.168.0.106', ssh_username='sanjay', ssh_password='san@9397', remote=False)  

In [ ]:
db_demand['Game_Name'].unique()

In [ ]:
db = db_demand[db_demand['Game_Name'] == 'finals_2019_07_6_3510']

In [ ]:
db = db[['Timeslot', 'Total_Consumption']]

y = db.set_index(['Timeslot'])
y.head(5)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(y, model='additive', period=168)
fig = decomposition.plot()
plt.show()

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 168) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

In [ ]:
# Optmal model parameters
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(1, 0, 1),
                                seasonal_order=(1, 1, 1, 168), 
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
pred = results.get_prediction(start=1000, dynamic=False)
pred_ci = pred.conf_int()

In [ ]:
ax = y.plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Retail_sold')
plt.legend()
plt.show()

In [ ]:
y_forecasted = pred.predicted_mean.tolist()
y_truth = y[1000:].values.tolist()
rmse = mean_squared_error(y_forecasted, y_truth, squared=False)
print('The Mean Squared Error is {}'.format(round(rmse, 2)))

In [ ]:
pred_uc = results.get_forecast(steps=24)
pred_ci = pred_uc.conf_int()
ax = y.plot(label='observed', figsize=(14, 4))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Sales')
plt.legend()
plt.show()